In [7]:
from Bio import Align
from Bio.Align import substitution_matrices

mylist = {'nacad': [], 'xirp1': [], '5730596b20rik': [], 'rif1': [], 'maz': [],
          'hic1': [], 'sdccag1': [], 'tmem107': [], 'srcin1': [], 'marcks': [],
          'senp6': [], 'phactr4': [], 'chrnb2': []}
mySRRList = ['SRR14676234', 'SRR14676248', 'SRR14676209', 'SRR14676235', 'SRR14676227']
for SRRfile in mySRRList:
    f = open(SRRfile + '/' + SRRfile + 'format3.txt', 'r')
    f2 = open(SRRfile + '/' + SRRfile + 'format2.txt', 'r')
    lines = f2.readlines()
    count = 0
    for row in f:
        fspSequences = {'nacad': 'VIYAPPP', 'xirp1': 'GKGPGGPP', '5730596b20rik': 
                       'GTLPPPP', 'rif1': 'AHTDKKKK', 'maz': 'PCTLLAP', 'hic1': 'DRTFPSPP',
                       'sdccag1': 'EAPKGKKK', 'tmem107': 'TQYFGMGG', 'srcin1': 'DEGMWPP', 'marcks': 'SSETPKKK',
                       'senp6': 'VKCSMKKK', 'phactr4': 'PWKWRKKK', 'chrnb2': 'VRTRPSPP'}
        for key in fspSequences:
            aligner = Align.PairwiseAligner()
            aligner.substitution_matrix = substitution_matrices.load("BLOSUM62")
            alignments = aligner.align(str(fspSequences[key]), row[1:len(row)-2])
            alignments = list(alignments)
            #print(count)

            #print("Number of alignments: %d" % len(alignments))

            alignment = alignments[0]

            #print("Score = %.1f" % alignment.score)

            #print(alignment)
            
            line = lines[count].split('\t')
            srrposition = line[0]
            cpgcount = line[1]
            mononucsequence = line[2]
            mylist[key].append((alignment.score, alignment, SRRfile, srrposition, cpgcount, mononucsequence))

            #if(len(mylist) <= 10):
                #mylist.append(str(alignment.score))
            #elif(min(mylist) < str(alignment.score)):
                #mylist.remove(min(mylist))
                #mylist.append(str(alignment.score))
        count+=1
    f2.close()
    f.close()
    print("Done.")

for key in mylist:
    while(len(mylist[key]) > 100):
        min = float('inf')
        for entry in mylist[key]:
            if(entry[0] < min):
                min = entry[0]
        for entry in mylist[key]:
            if(entry[0] == min):
                mylist[key].remove(entry)
                break
    print(key, "done.")
f1 = open('fileOutScore.txt', 'a')
for key in mylist:
    for entry in mylist[key]:
        MSI_Val = entry[0] + (int(entry[4])*1000) + (len(entry[5])*100)
        f1.write(str(key) + '\t' + str(MSI_Val) + '\t' + str(entry[1]).split('\n')[0] + '\t' + str(entry[1]).split('\n')[2] + '\t' + str(entry[2]) + '\t' + str(entry[3]) + '\t' + str(entry[5]))
f1.close()

Done.
Done.
Done.
Done.
Done.
nacad done.
xirp1 done.
5730596b20rik done.
rif1 done.
maz done.
hic1 done.
sdccag1 done.
tmem107 done.
srcin1 done.
marcks done.
senp6 done.
phactr4 done.
chrnb2 done.
